<!-- <h1 style="text-align: center;">
    ATSC 409 Project: 1D Radiative Model
</h1>
<h5 style="text-align: center;">
    - Anirudh Duggal, Clinton Macadam, and Hanrui Duan
</h5> -->
# ATSC 409 Project: 1D Radiative Model
##### Anirudh Duggal, Clinton Macadam, and Hanrui Duan

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Runge-Kutta 4th order
def rk4(f, x0, y0, x1, n):
    h = (x1 - x0) / n
    x = np.arange(x0, x1, h)
    y = np.zeros((n + 1))
    y[0] = y0
    for i in range(n):
        k1 = h * f(x[i], y[i])
        k2 = h * f(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * f(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * f(x[i] + h, y[i] + k3)
        y[i + 1] = y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6
    return x, y

In [15]:
rk4(lambda x, y: y, 0, 1, 1, 100)

(array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
        0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
        0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
        0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
        0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
        0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
        0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
        0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
        0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
        0.99]),
 array([1.        , 1.01005017, 1.02020134, 1.03045453, 1.04081077,
        1.0512711 , 1.06183655, 1.07250818, 1.08328707, 1.09417428,
        1.10517092, 1.11627807, 1.12749685, 1.13882838, 1.1502738 ,
        1.16183424, 1.17351087, 1.18530485, 1.19721736, 1.2092496 ,
        1.22140276, 1.23367806, 1.24607673, 1.